In [ ]:
!pip install -q google-genai

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set


# Agent A: Personal Assistant

Our first real agent! It has two tools:
- **Calculator** — evaluates math expressions (a real tool, not hardcoded!)
- **Clock** — returns the current date and time

This uses everything from exercises 1-6: API calls, system instructions, tools, and the agentic loop.

## Step 1: Setup

In [ ]:
from google import genai
from google.genai import types
from datetime import datetime

client = genai.Client()

## Step 2: Define the system instruction

Notice we tell the model to **always** use the calculator for math. Without this, it might try to do arithmetic itself (and get it wrong).

In [ ]:
SYSTEM_PROMPT = """You are a helpful personal assistant. You have access to a calculator and a clock.
Use the calculator for any math — never do arithmetic in your head.
Be concise and friendly."""

## Step 3: Define tools and the dispatcher

The calculator uses Python's `eval()` on a restricted character set. The clock uses `datetime.now()`.

These are **real** tools — they compute actual results, not hardcoded strings.

In [ ]:
def calculator(expression: str) -> str:
    """Evaluate a mathematical expression. Supports +, -, *, /, **, parentheses, and common math."""
    pass


def get_time() -> str:
    """Get the current date, day of week, and time."""
    pass


def call_tool(name, args):
    if name == "calculator":
        try:
            allowed = set("0123456789+-*/.() ")
            expr = args["expression"]
            if all(c in allowed for c in expr):
                return str(eval(expr))
            else:
                return "Error: expression contains invalid characters"
        except Exception as e:
            return f"Error: {e}"
    elif name == "get_time":
        return datetime.now().strftime("%A, %Y-%m-%d %H:%M:%S")
    return "Unknown tool"


config = types.GenerateContentConfig(
    system_instruction=SYSTEM_PROMPT,
    tools=[calculator, get_time],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True),
)

## Step 4: Run the agent

Same agentic loop from exercise 6, now with a system instruction and real tools.

In [ ]:
contents = [
    types.Content(role="user", parts=[types.Part.from_text(text="What is 2847 divided by 13? Also what day of the week is it?")]),
]

while True:
    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=contents,
        config=config,
    )

    model_content = response.candidates[0].content
    contents.append(model_content)

    function_calls = [part.function_call for part in model_content.parts if part.function_call]

    if function_calls:
        tool_response_parts = []
        for fc in function_calls:
            result = call_tool(fc.name, dict(fc.args))
            print(f"  [{fc.name}] {dict(fc.args)} → {result}")
            tool_response_parts.append(
                types.Part.from_function_response(name=fc.name, response={"output": result})
            )
        contents.append(types.Content(role="user", parts=tool_response_parts))
    else:
        print(f"\nAssistant: {response.text}")
        break

## Try it yourself

Change the user message above. Some ideas:
- "What is 15% tip on a 84.50 dollar bill?"
- "What time is it right now?"
- "If I earn 45 dollars per hour and work 37.5 hours a week, what's my monthly income?"

For an interactive chat version, run:
```bash
uv run python foundation/gemini/07_agent_personal/agent_personal.py
```